In [1]:
from flask import jsonify
from connect_db import execute_query
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pickle
from app import app  # Importer l'application Flask

c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.3.0 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\antho\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to un

In [2]:
with app.app_context():
    query = "SELECT * FROM clean_olympic_results"
    data = execute_query(query)
    json_data = jsonify(data)
    
    # Extraire les données JSON de l'objet Response
    real_data = json_data.get_json()
    real_data

Connexion réussie
Connexion fermée


In [3]:
results = pd.DataFrame(real_data)
results.head()

,athletes,country_3_letter_code,country_code,country_name,discipline_title,event_title,id,medal_type,participant_type,rank_equal,rank_position,slug_game
0,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",ESP,ES,Spain,Shooting,Trap Mixed Team,1,GOLD,GameTeam,0,1,tokyo-2020
1,"[('Alessandra PERILLI', 'https://olympics.com/...",SMR,SM,San Marino,Shooting,Trap Mixed Team,2,SILVER,GameTeam,0,2,tokyo-2020
2,"[('Madelynn Ann BERNAU', 'https://olympics.com...",USA,US,United States of America,Shooting,Trap Mixed Team,3,BRONZE,GameTeam,0,3,tokyo-2020
3,"[('Zuzana STEFECEKOVA', 'https://olympics.com/...",SVK,SK,Slovakia,Shooting,Trap Mixed Team,4,AUCUNE,GameTeam,0,4,tokyo-2020
4,"[('Yukie NAKAYAMA', 'https://olympics.com/en/a...",JPN,JP,Japan,Shooting,Trap Mixed Team,5,AUCUNE,GameTeam,0,5,tokyo-2020


In [4]:
category_list = results['event_title'].unique()
category_list   

array(['Trap Mixed Team', '50m Rifle 3 Positions women',
       '50m Rifle 3 Positions Men', ..., 'army rifle 300m men',
       'Unlimited Class, Greco-Roman Men',
       'heavyweight - one hand lift men'], dtype=object)

In [5]:
for cat in category_list:
    if "women" in cat.lower():
        results['event_title'].replace(cat, 'women', inplace=True)
    elif ' men' in cat.lower():
        results['event_title'].replace(cat, 'Man', inplace=True)
    elif 'Men' in cat:
        results['event_title'].replace(cat, 'Man', inplace=True)
    elif 'Me' in cat:
        results['event_title'].replace(cat, 'Man', inplace=True)
    elif ' M' in cat:
        results['event_title'].replace(cat, 'Man', inplace=True)
    elif 'mixed team' in cat.lower():
        results['event_title'].replace(cat, 'mixedT', inplace=True)
    elif 'individual' in cat.lower():
        results['event_title'].replace(cat, 'individual', inplace=True)
    elif 'one' in cat.lower():
        results['event_title'].replace(cat, 'individual', inplace=True)
    elif 'single' in cat.lower():
        results['event_title'].replace(cat, 'individual', inplace=True)
    elif ' Ind' in cat:
        results['event_title'].replace(cat, 'individual', inplace=True)
    elif 'mixed' in cat.lower():
        results['event_title'].replace(cat, 'mixedT', inplace=True)
    elif 'team' in cat.lower():
        results['event_title'].replace(cat, 'team', inplace=True)
    elif 'group' in cat.lower():
        results['event_title'].replace(cat, 'team', inplace=True)
    elif 'tea' in cat.lower():
        results['event_title'].replace(cat, 'team', inplace=True)
    elif ' T' in cat:
        results['event_title'].replace(cat, 'team', inplace=True)
    else:
        results['event_title'].replace(cat, 'team', inplace=True)

In [6]:
results['slug_game'] = results['slug_game'].apply(lambda x: int(x.split('-')[-1]))

In [7]:
i=0
for cat in results['event_title'].unique():
    results.loc[results['event_title'] == cat, 'cat'] = i
    i+=1
for sport in results['discipline_title'].unique():
    results.loc[results['discipline_title'] == sport, 'sport'] = i
    i+=1  
for country in results['country_name'].unique():
    results.loc[results['country_name'] == country, 'country'] = i
    i+=1
results

,athletes,country_3_letter_code,country_code,country_name,discipline_title,event_title,id,medal_type,participant_type,rank_equal,rank_position,slug_game,cat,sport,country
0,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",ESP,ES,Spain,Shooting,Man,1,GOLD,GameTeam,0,1,2020,0.0,5.0,76.0
1,"[('Alessandra PERILLI', 'https://olympics.com/...",SMR,SM,San Marino,Shooting,Man,2,SILVER,GameTeam,0,2,2020,0.0,5.0,77.0
2,"[('Madelynn Ann BERNAU', 'https://olympics.com...",USA,US,United States of America,Shooting,Man,3,BRONZE,GameTeam,0,3,2020,0.0,5.0,78.0
3,"[('Zuzana STEFECEKOVA', 'https://olympics.com/...",SVK,SK,Slovakia,Shooting,Man,4,AUCUNE,GameTeam,0,4,2020,0.0,5.0,79.0
4,"[('Yukie NAKAYAMA', 'https://olympics.com/en/a...",JPN,JP,Japan,Shooting,Man,5,AUCUNE,GameTeam,0,5,2020,0.0,5.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118350,,AUS,AU,Australia,Equestrian Jumping,individual,118351,AUCUNE,Athlete,0,0,1976,4.0,54.0,81.0
118351,,AUT,AT,Austria,Equestrian Jumping,individual,118352,AUCUNE,Athlete,0,0,1976,4.0,54.0,124.0
118352,,BOL,BO,Bolivia,Equestrian Jumping,individual,118353,AUCUNE,Athlete,0,0,1976,4.0,54.0,236.0
118353,,ESP,ES,Spain,Equestrian Jumping,individual,118354,AUCUNE,Athlete,0,0,1976,4.0,54.0,76.0


In [8]:
json_data = {
    'country': {},
    'sport': {},
    'cat': {}
}
json_data

{'country': {}, 'sport': {}, 'cat': {}}

In [9]:
for i in range(len(results)):
    if results['country'][i] not in json_data['country']:
        json_data['country'][results['country'][i]] = results['country_name'][i]
    if results['sport'][i] not in json_data['sport']:
        json_data['sport'][results['sport'][i]] = results['discipline_title'][i]
    if results['cat'][i] not in json_data['cat']:   
        json_data['cat'][results['cat'][i]] = results['event_title'][i]
json_data

{'country': {76.0: 'Spain',
  77.0: 'San Marino',
  78.0: 'United States of America',
  79.0: 'Slovakia',
  80.0: 'Japan',
  81.0: 'Australia',
  82.0: "People's Republic of China",
  83.0: 'Great Britain',
  84.0: 'ROC',
  85.0: 'Italy',
  86.0: 'Mexico',
  87.0: 'Egypt',
  88.0: 'Belarus',
  89.0: 'Islamic Republic of Iran',
  90.0: 'Switzerland',
  91.0: 'Norway',
  92.0: 'Serbia',
  93.0: 'Germany',
  94.0: 'Indonesia',
  95.0: 'Czech Republic',
  96.0: 'Mongolia',
  97.0: 'Puerto Rico',
  98.0: 'Poland',
  99.0: 'Republic of Korea',
  100.0: 'Uzbekistan',
  101.0: 'Malaysia',
  102.0: 'Slovenia',
  103.0: 'Hungary',
  104.0: 'Cuba',
  105.0: 'France',
  106.0: 'India',
  107.0: 'Denmark',
  108.0: 'Croatia',
  109.0: 'Ukraine',
  110.0: 'Oman',
  111.0: 'Argentina',
  112.0: 'Turkey',
  113.0: 'Kazakhstan',
  114.0: 'Venezuela',
  115.0: 'Lithuania',
  116.0: 'Estonia',
  117.0: 'Colombia',
  118.0: 'Azerbaijan',
  119.0: 'Peru',
  120.0: 'Pakistan',
  121.0: 'Thailand',
  122.0: 

In [10]:
import json

with open("dataModelSport.json", "w") as f:
    json.dump(json_data, f)

In [11]:
df = results.groupby(['discipline_title','cat', 'slug_game', 'country_name', 'country', 'medal_type']).size().unstack(fill_value=0).reset_index()
# df['slug_game'] = df['slug_game'].apply(lambda x: int(x.split('-')[-1]))
df

medal_type,discipline_title,cat,slug_game,country_name,country,AUCUNE,BRONZE,GOLD,SILVER
0,3x3 Basketball,0.0,2020,Belgium,131.0,1,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,1,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,0,1,0
3,3x3 Basketball,0.0,2020,Netherlands,179.0,1,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
32717,Wrestling,1.0,2020,Sweden,175.0,2,0,0,0
32718,Wrestling,1.0,2020,Tunisia,160.0,4,0,0,0
32719,Wrestling,1.0,2020,Turkey,112.0,1,1,0,0
32720,Wrestling,1.0,2020,Ukraine,109.0,3,2,0,0


In [12]:
df.drop('AUCUNE', axis=1, inplace=True)
df

medal_type,discipline_title,cat,slug_game,country_name,country,BRONZE,GOLD,SILVER
0,3x3 Basketball,0.0,2020,Belgium,131.0,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,1,0
3,3x3 Basketball,0.0,2020,Netherlands,179.0,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,0,0,0
...,...,...,...,...,...,...,...,...
32717,Wrestling,1.0,2020,Sweden,175.0,0,0,0
32718,Wrestling,1.0,2020,Tunisia,160.0,0,0,0
32719,Wrestling,1.0,2020,Turkey,112.0,1,0,0
32720,Wrestling,1.0,2020,Ukraine,109.0,2,0,0


In [13]:
df['MEDALS'] = df['SILVER'] + df['BRONZE'] + df['GOLD']
df

medal_type,discipline_title,cat,slug_game,country_name,country,BRONZE,GOLD,SILVER,MEDALS
0,3x3 Basketball,0.0,2020,Belgium,131.0,0,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,0,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,1,0,1
3,3x3 Basketball,0.0,2020,Netherlands,179.0,0,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
32717,Wrestling,1.0,2020,Sweden,175.0,0,0,0,0
32718,Wrestling,1.0,2020,Tunisia,160.0,0,0,0,0
32719,Wrestling,1.0,2020,Turkey,112.0,1,0,0,1
32720,Wrestling,1.0,2020,Ukraine,109.0,2,0,0,2


In [14]:
df_data_2020 = df[df['slug_game'] == 2020].copy()
df_data_2020

medal_type,discipline_title,cat,slug_game,country_name,country,BRONZE,GOLD,SILVER,MEDALS
0,3x3 Basketball,0.0,2020,Belgium,131.0,0,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,0,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,1,0,1
3,3x3 Basketball,0.0,2020,Netherlands,179.0,0,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
32717,Wrestling,1.0,2020,Sweden,175.0,0,0,0,0
32718,Wrestling,1.0,2020,Tunisia,160.0,0,0,0,0
32719,Wrestling,1.0,2020,Turkey,112.0,1,0,0,1
32720,Wrestling,1.0,2020,Ukraine,109.0,2,0,0,2


In [15]:
df_man = df[df['cat'] == 0]
df_man

medal_type,discipline_title,cat,slug_game,country_name,country,BRONZE,GOLD,SILVER,MEDALS
0,3x3 Basketball,0.0,2020,Belgium,131.0,0,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,0,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,1,0,1
3,3x3 Basketball,0.0,2020,Netherlands,179.0,0,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
32545,Wrestling,0.0,2020,Tunisia,160.0,0,0,0,0
32546,Wrestling,0.0,2020,Turkey,112.0,2,0,0,2
32547,Wrestling,0.0,2020,Ukraine,109.0,0,1,1,2
32548,Wrestling,0.0,2020,United States of America,78.0,2,2,1,5


In [16]:
df_man.rename(columns={'slug_game': 'année', 'discipline_title': 'sport', 'country_name': 'pays', 'MEDALS': 'médailles'}, inplace=True)
df_man

C:\Users\antho\AppData\Local\Temp\ipykernel_14420\3264097238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_man.rename(columns={'slug_game': 'année', 'discipline_title': 'sport', 'country_name': 'pays', 'MEDALS': 'médailles'}, inplace=True)


medal_type,sport,cat,année,pays,country,BRONZE,GOLD,SILVER,médailles
0,3x3 Basketball,0.0,2020,Belgium,131.0,0,0,0,0
1,3x3 Basketball,0.0,2020,Japan,80.0,0,0,0,0
2,3x3 Basketball,0.0,2020,Latvia,157.0,0,1,0,1
3,3x3 Basketball,0.0,2020,Netherlands,179.0,0,0,0,0
4,3x3 Basketball,0.0,2020,People's Republic of China,82.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
32545,Wrestling,0.0,2020,Tunisia,160.0,0,0,0,0
32546,Wrestling,0.0,2020,Turkey,112.0,2,0,0,2
32547,Wrestling,0.0,2020,Ukraine,109.0,0,1,1,2
32548,Wrestling,0.0,2020,United States of America,78.0,2,2,1,5


In [25]:
df_man = df_man.drop(df_man[(df_man['pays'] == 'Soviet Union') | (df_man['pays'] == 'MIX') ].index)

In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

# Préparation des données
X = df_man[['année', 'sport', 'pays']]
y = df_man['médailles']

# Encodage des variables catégorielles
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Création du modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Erreur absolue moyenne : {mae}')

# Fonction pour prédire le nombre de médailles par pays pour une année et un sport donnés
def predire_medailles_par_pays(annee, sport):
    # Filtrer les pays pour l'année et le sport donnés
    pays_list = df_man['pays'].unique()
    predictions = []
    for pays in pays_list:
        input_data = pd.DataFrame({'année': [annee], 'sport': [sport], 'pays': [pays]})
        
        # Utiliser les mêmes colonnes que lors de l'encodage initial
        input_encoded = encoder.transform(input_data[['année', 'sport', 'pays']])
        
        prediction = model.predict(input_encoded)
        predictions.append((pays, prediction[0]))
    
    # Trier les pays par nombre de médailles prédites
    predictions.sort(key=lambda x: x[1], reverse=True)
    print(predictions)
    return predictions

# Exemple de prédiction
annee = 2024
sport = 'Rugby'
predictions = predire_medailles_par_pays(annee, sport)

# Affichage du pays avec le plus de médailles prédites
print(f'Pays avec le plus de médailles en {sport} en {annee} : {predictions[0][0]} avec {predictions[0][1]} médailles')

Erreur absolue moyenne : 0.4669126111983255
[('Fiji', 0.75), ('Australasia', 0.75), ('South Africa', 0.66), ('Haiti', 0.66), ('Gabon', 0.66), ('Luxembourg', 0.65), ('Philippines', 0.65), ('Puerto Rico', 0.65), ('Costa Rica', 0.65), ('Ireland', 0.65), ('Thailand', 0.65), ('Malta', 0.65), ('Zimbabwe', 0.65), ('Bhutan', 0.65), ('Chinese Taipei', 0.65), ('Colombia', 0.65), ('Hong Kong, China', 0.65), ('Monaco', 0.65), ('Portugal', 0.65), ('Saudi Arabia', 0.65), ('Solomon Islands', 0.65), ('Cyprus', 0.65), ('Guam', 0.65), ('San Marino', 0.65), ('American Samoa', 0.65), ('El Salvador', 0.65), ('Mauritius', 0.65), ('Vanuatu', 0.65), ("Lao People's Democratic Republic", 0.65), ('Tonga', 0.65), ('Myanmar', 0.65), ('Samoa', 0.65), ('Bangladesh', 0.65), ('Ivory Coast', 0.65), ('Republic of Moldova', 0.65), ('Venezuela', 0.65), ('Libya', 0.65), ('Malawi', 0.65), ('Nepal', 0.65), ('Virgin Islands, US', 0.65), ('Bohemia', 0.65), ('Newfoundland', 0.65), ('Ecuador', 0.65), ('Peru', 0.65), ('Guyana', 0

In [27]:
# Importation des bibliothèques
import pickle
import os
# Création d'un dossier s'il n'existe pas
if not os.path.exists('./models'):
    os.mkdir('models')
    pass
# Sauvegarde du modèle
pickle.dump(model, open('./models/modelsportmen.pkl','wb'))

In [ ]:
df_woman = df[df['cat'] == 1]
df_woman    

medal_type,sport,cat,slug_game,country,BRONZE,GOLD,SILVER
1147,5.0,1.0,1984,76.0,0,0,0
1148,5.0,1.0,1984,78.0,1,1,1
1149,5.0,1.0,1984,80.0,0,0,0
1150,5.0,1.0,1984,81.0,1,0,0
1151,5.0,1.0,1984,82.0,1,1,0
...,...,...,...,...,...,...,...
32694,67.0,1.0,1908,175.0,0,0,0
32695,67.0,1.0,1920,78.0,1,0,0
32696,67.0,1.0,1920,83.0,0,0,0
32697,67.0,1.0,1920,91.0,0,0,0


In [ ]:
df_team = df[df['cat'] == 2]
df_team

medal_type,sport,cat,slug_game,country,BRONZE,GOLD,SILVER
1678,5.0,2.0,1908,78.0,0,1,0
1679,5.0,2.0,1908,83.0,0,0,1
1680,5.0,2.0,1908,91.0,0,0,0
1681,5.0,2.0,1908,105.0,0,0,0
1682,5.0,2.0,1908,107.0,0,0,0
...,...,...,...,...,...,...,...
28724,48.0,2.0,2020,88.0,0,0,0
28725,48.0,2.0,2020,109.0,0,0,0
28726,48.0,2.0,2020,122.0,0,0,0
28727,48.0,2.0,2020,152.0,0,1,0


In [ ]:
df_mixed = df[df['cat'] == 3]
df_mixed

medal_type,sport,cat,slug_game,country,BRONZE,GOLD,SILVER
1678,5.0,2.0,1908,78.0,0,1,0
1679,5.0,2.0,1908,83.0,0,0,1
1680,5.0,2.0,1908,91.0,0,0,0
1681,5.0,2.0,1908,105.0,0,0,0
1682,5.0,2.0,1908,107.0,0,0,0
...,...,...,...,...,...,...,...
28724,48.0,2.0,2020,88.0,0,0,0
28725,48.0,2.0,2020,109.0,0,0,0
28726,48.0,2.0,2020,122.0,0,0,0
28727,48.0,2.0,2020,152.0,0,1,0


In [ ]:
df_indiv = df[df['cat'] == 4]
df_indiv

medal_type,sport,cat,slug_game,country,BRONZE,GOLD,SILVER
1958,5.0,4.0,1972,76.0,0,0,0
1959,5.0,4.0,1972,78.0,0,0,1
1960,5.0,4.0,1972,80.0,0,0,0
1961,5.0,4.0,1972,81.0,0,0,0
1962,5.0,4.0,1972,83.0,0,0,0
...,...,...,...,...,...,...,...
32531,59.0,4.0,2016,266.0,0,0,0
32677,66.0,4.0,1920,105.0,0,0,1
32678,66.0,4.0,1920,131.0,1,1,0
32679,66.0,4.0,1920,175.0,0,0,0


In [ ]:
# Séparation des caractéristiques et de la cible
mlX, mly = df[['year', 'countryId']].values, df[['gold', 'silver', 'bronze']].values

KeyError: "None of [Index(['year', 'countryId'], dtype='object', name='medal_type')] are in the [columns]"